In [ ]:
!pip install livelossplot --quiet

In [ ]:
# Importing the libraries
import os
import shutil
import glob
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import cv2
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random
from random import seed
from livelossplot import PlotLossesKeras
import math

In [ ]:
# Importing the Training Dataset
df_train = pd.read_csv("../input/jpeg-melanoma-384x384/train.csv")
df_train.head()

In [ ]:
df_train.info()

In [ ]:
# Importing the Test Dataset
df_test = pd.read_csv("../input/jpeg-melanoma-384x384/test.csv")
df_test.head()

In [ ]:
# Creating a config class to store all the configurations
class config:
    
    # Image and Tabular data paths
    DIRECTORY_PATH = "../input/jpeg-melanoma-384x384/"
    TRAINING_SAMPLES_FOLDER = DIRECTORY_PATH + "train/"
    TESTING_SAMPLES_FOLDER = DIRECTORY_PATH + "test/"
    TRAIN_FULL_DATA = DIRECTORY_PATH + "train.csv"
    TEST_FULL_DATA = DIRECTORY_PATH + "test.csv"
    
    # New directory path for image data
    WORK_DIRECTORY = "dataset/"
    TRAIN_IMAGES_FOLDER = WORK_DIRECTORY + "training_set/"
    TEST_IMAGES_FOLDER = WORK_DIRECTORY + "test_set/"
    VALIDATION_IMAGES_FOLDER = WORK_DIRECTORY + "validation_set/"
    
    # Input parameters for data preprocessing
    TARGET_NAME = "target"
    TRAIN_SIZE = 0.80
    VALIDATION_SIZE = 0.10
    TEST_SIZE = 0.10
    SEED = 42
    
    # Tensorflow settings for model training
    IMAGE_HEIGHT = 299
    IMAGE_WIDTH = 299
    NO_CHANNELS = 3
    BATCH_SIZE = 64
    EPOCHS = 20
    DROPOUT = 0.5
    LEARNING_RATE = 0.01
    PATIENCE = 5

In [ ]:
# Checking the files along with the number of samples
print(os.listdir(config.DIRECTORY_PATH))
print(len(os.listdir(config.TRAINING_SAMPLES_FOLDER)), "Training Samples")
print(len(os.listdir(config.TESTING_SAMPLES_FOLDER)), "Testing Samples")

In [ ]:
# Creating folders for training and validation data
dataset_home = "./dataset/"
subdirs = ["training_set/", "test_set/", "validation_set/"]
for subdir in subdirs:
    labeldirs = ["benign", "malignant"]
    for labeldir in labeldirs:
        newdir = dataset_home + subdir + labeldir
        os.makedirs(newdir, exist_ok=True)

In [ ]:
print(os.listdir("./dataset"))

In [ ]:
# Splitting the dataset into train, test and validation set

test_examples = train_examples = validation_examples = 0
seed(config.SEED)

for record in open(config.TRAIN_FULL_DATA).readlines()[1:]:
    split_record = record.split(",")
    image_name = split_record[0]
    target = split_record[7]
    
    random_num = random.random()
    
    if random_num < config.TRAIN_SIZE:
        destination = config.TRAIN_IMAGES_FOLDER
        train_examples += 1
        
    elif random_num < 0.9:
        destination = config.VALIDATION_IMAGES_FOLDER
        validation_examples += 1
        
    else:
        destination = config.TEST_IMAGES_FOLDER
        test_examples += 1
        
    if target == "0":
        shutil.copy(
            config.TRAINING_SAMPLES_FOLDER + image_name + ".jpg",
            destination + "benign/" + image_name + ".jpg"
        )
    
    elif target == "1":
        shutil.copy(
            config.TRAINING_SAMPLES_FOLDER + image_name + ".jpg",
            destination + "malignant/" + image_name + ".jpg"
        )

print(f"Number of training examples: {train_examples}")
print(f"Number of test examples: {test_examples}")
print(f"Number of validation examples: {validation_examples}")

In [ ]:
# Preparing the data and performing Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=(0.95, 0.95),
    rotation_range=15,
    horizontal_flip=True,
    vertical_flip=True,
    data_format="channels_last",
    dtype=tf.float32
)

validation_datagen = ImageDataGenerator(
    rescale=1./255,
    dtype=tf.float32
)

train_generator = train_datagen.flow_from_directory(
    directory=config.TRAIN_IMAGES_FOLDER,
    target_size=(config.IMAGE_HEIGHT, config.IMAGE_WIDTH),
    color_mode="rgb",
    batch_size=config.BATCH_SIZE,
    class_mode="binary",
    shuffle=True
)

validation_generator = validation_datagen.flow_from_directory(
    directory=config.VALIDATION_IMAGES_FOLDER,
    target_size=(config.IMAGE_HEIGHT, config.IMAGE_WIDTH),
    color_mode="rgb",
    batch_size=config.BATCH_SIZE,
    class_mode="binary",
    shuffle=True
)

In [ ]:
# Creating an exponential learning rate scheduler
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1**(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(lr0=config.LEARNING_RATE, s=20)
lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)

In [ ]:
# Using callbacks to save model parameters and perform early stopping
checkpoint_cb = keras.callbacks.ModelCheckpoint("melanoma.h5", save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=config.PATIENCE, restore_best_weights=True)

In [ ]:
# Calculating the different step size for the model while training
STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
STEP_SIZE_VALIDATION = validation_generator.n // validation_generator.batch_size

In [ ]:
# Metrics to use for compiling the model
METRICS = [keras.metrics.AUC(name="auc")]

In [ ]:
# Calculating the number of benign/malignant images
total_images = train_examples + validation_examples
benign_images = len(os.listdir(config.TRAIN_IMAGES_FOLDER + "benign")) + len(os.listdir(config.VALIDATION_IMAGES_FOLDER + "benign"))
malignant_images = len(os.listdir(config.TRAIN_IMAGES_FOLDER + "malignant")) + len(os.listdir(config.VALIDATION_IMAGES_FOLDER + "malignant"))

In [ ]:
# Building a transfer learning model using Keras
xception = keras.applications.xception.Xception(input_shape=(config.IMAGE_HEIGHT, config.IMAGE_WIDTH, config.NO_CHANNELS),
                                       weights="imagenet", include_top=False)
xception.trainable = False

xcept_model = keras.models.Sequential([
    xception,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(8, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
# Compiling the transfer learning model
xcept_model.compile(optimizer=keras.optimizers.Adam(), 
              loss=keras.losses.BinaryCrossentropy(),
              metrics=METRICS
             )

In [ ]:
# Training the CNN model
xcept_history = xcept_model.fit(train_generator, steps_per_epoch=STEP_SIZE_TRAIN, epochs=config.EPOCHS,
                  validation_data=validation_generator, validation_steps=STEP_SIZE_VALIDATION,
                  callbacks=[early_stopping_cb, checkpoint_cb, lr_scheduler, PlotLossesKeras()]
                 )

In [ ]:
# Evaluating the model on Validation Dataset
xcept_model.evaluate(validation_generator, steps=STEP_SIZE_VALIDATION)

In [ ]:
# Creating a test generator for test data
test_datagen = ImageDataGenerator(
    rescale=1./255,
    dtype=tf.float32
)

test_generator = test_datagen.flow_from_directory(
    directory=config.TEST_IMAGES_FOLDER,
    target_size=(config.IMAGE_HEIGHT, config.IMAGE_WIDTH),
    color_mode="rgb",
    batch_size=config.BATCH_SIZE,
    class_mode="binary",
    shuffle=False
)

In [ ]:
STEP_SIZE_TEST = test_generator.n // test_generator.batch_size

In [ ]:
# Getting the actual classes of the test dataset
y_test = np.array([])
num_batches = 0
for _, y in test_generator:
    y_test = np.append(y_test, y)
    num_batches += 1
    if num_batches == math.ceil(test_examples / config.BATCH_SIZE):
        break
y_test

In [ ]:
# Predicting output on the test dataset
y_pred = xcept_model.predict(test_generator)
y_pred

In [ ]:
# Computing the TPR and FPR values from the roc curve
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred)

In [ ]:
# Plotting the ROC curve
def plot_roc_curve (fpr, tpr, label = None):
    plt.plot(fpr, tpr, linewidth = 2, label = label)
    plt.plot([0,1], [0,1], 'k--') # Dashed diagonal
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate (Recall)")
    plt.grid()
    
plot_roc_curve(fpr, tpr)
plt.show()

In [ ]:
# Evaluating the model on test dataset
xcept_model.evaluate(test_generator, steps=STEP_SIZE_TEST)

In [ ]:
# Saving the best model after training
xcept_model.save("final_melanoma_model.h5")